In [ ]:
import requests
import pandas as pd
import time

def fetch_klines(symbol, interval, start_time, end_time):
    url = 'https://fapi.binance.com/fapi/v1/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1500  # Max limit per request
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def get_historical_klines(symbol, interval, start_str, end_str):
    start_time = int(pd.to_datetime(start_str).timestamp() * 1000)
    end_time = int(pd.to_datetime(end_str).timestamp() * 1000)
    all_klines = []
    while start_time < end_time:
        klines = fetch_klines(symbol, interval, start_time, end_time)
        if not klines:
            break
        all_klines.extend(klines)
        # Update start_time to the next timestamp to avoid duplicate data
        start_time = klines[-1][6] + 1
        time.sleep(0.5)  # Sleep to respect API rate limits
    return all_klines

# Usage example
symbol = 'BTCUSDT'
interval = '1m'
start_str = '2022-12-31'
end_str = '2025-01-01'

klines = get_historical_klines(symbol, interval, start_str, end_str)

# Convert to DataFrame
columns = [
    'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close Time', 'Quote Asset Volume', 'Number of Trades',
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
]
df = pd.DataFrame(klines, columns=columns)

# Convert timestamps to datetime
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

# ----------------------------

columns_to_remove = ['Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
df.drop(columns=columns_to_remove, inplace=True)

df['Open Time'] = pd.to_datetime(df['Open Time'])

df['Date'] = df['Open Time'].dt.date
df['Time'] = df['Open Time'].dt.time

df.drop(columns=['Open Time'], inplace=True)

df.rename(columns={
    'Date': 'date',
    'Time': 'time',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

df = df[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]

# ----------------------------

# Save to CSV
df.to_csv('BTCUSDT_perpetual_1m_2023_2024.csv', index=False)

print("Data saved to BTCUSDT_perpetual_1m.csv")
